In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

from scipy.stats import norm
from scipy.stats import multivariate_normal

In [3]:
#load data into memory
data = pd.DataFrame(data = np.genfromtxt('ps2.dat', delimiter='  '), columns=['y','x1','x2','z'])

print data.mean()

y      0.568393
x1    42.537849
x2    12.286853
z      9.029880
dtype: float64


# Part a

An economic story where $x_{2i}$ is correlated with $\epsilon_i$ involves simultaneity between the decision of an education level and years they want to work. Women who intend to stay in the labor force longer may have select more education.

# Part b

# Part c

$\rho$ represents the correlation between the two error terms. The intrument is relevant it should be non-zero. Since, one would expect parents education to be positively related to your education, you would expect $\rho$ to be positive.

# Part d

In [9]:
#run ols first to help fit

model_d_stage1 = sm.OLS(data['x2'],sm.add_constant(data[['x1','z']]) )
model_d_stage1_fit = model_d_stage1.fit()
data['eta'] = model_d_stage1_fit.resid
T3, T4, T5 = model_d_stage1_fit.params

print model_d_stage1_fit.summary()

                            OLS Regression Results                            
Dep. Variable:                     x2   R-squared:                       0.245
Model:                            OLS   Adj. R-squared:                  0.243
Method:                 Least Squares   F-statistic:                     121.7
Date:                Thu, 25 Oct 2018   Prob (F-statistic):           1.63e-46
Time:                        17:03:49   Log-Likelihood:                -1582.8
No. Observations:                 753   AIC:                             3172.
Df Residuals:                     750   BIC:                             3185.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.1255      0.493     18.507      0.0

In [14]:
class part_d(GenericLikelihoodModel):
    """class for evaluating question 1 part b"""
    
    def nloglikeobs(self, params, v=False):
        
        t0,t1,t2,t3,t4,t5,rho,sigma = params

        y,x2 = self.endog.transpose()
        x1,z,eta = self.exog.transpose()
        
        mu_epsilon = (rho/sigma**2)*eta
        var_epsilon = np.sqrt(abs(1 - (rho/sigma)**2))
        
        #pr(eta | ... )
        recalc_eta = x2 - t3 - t4*x1 - t5*z
        pr_eta = norm(0,sigma).pdf(recalc_eta)/sigma
        
        #pr(y|x2 ... )
        gamma = t0 + t2*t3 + (t1 + t2*t4)*x1 + t2*t5*z + t2*eta
        
        pr_epsilon = 1 - norm(mu_epsilon,var_epsilon).cdf(-gamma)
        #if v: print pr_epsilon.max(), pr_epsilon.min(), pr_epsilon.mean(), '\n'
        
        #likelihood = np.log( pr_eta * np.maximum(pr_epsilon,.0001) )
        likelihood = np.log( pr_epsilon*pr_eta )

        
        if v: raise Exception('Stop drop and roll')
        
        return -( likelihood.sum() ) 
    
    
    def fit(self, start_params=None, maxiter=500, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        if start_params == None:
            start_params = [-.4,.3,.2,T3,T4,T5,.12,1.33]
            
        return super(part_d, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)

    
model_d = part_d(data[['y','x2']],data[['x1','z','eta']])

result_d = model_d.fit()
print(result_d.summary(xname=['theta_0', 'theta_1', 'theta_2',
                              'theta_3','theta_4','theta_5',
                              'rho', 'sigma']))

Optimization terminated successfully.
         Current function value: 2.591876
         Iterations: 113
         Function evaluations: 204
                                part_d Results                                
Dep. Variable:            ['y', 'x2']   Log-Likelihood:                -1951.7
Model:                         part_d   AIC:                             3909.
Method:            Maximum Likelihood   BIC:                             3923.
Date:                Thu, 25 Oct 2018                                         
Time:                        17:41:25                                         
No. Observations:                 753                                         
Df Residuals:                     750                                         
Df Model:                           2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
